In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\jaesc2\GitHub\skforecast


In [2]:
import re
import pytest
import numpy as np
import pandas as pd
from sklearn.exceptions import NotFittedError
from skforecast.preprocessing import RollingFeatures
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import HistGradientBoostingRegressor
from lightgbm import LGBMRegressor

In [29]:
# Create and fit forecaster
# ==============================================================================
rolling = RollingFeatures(stats= ['mean', 'median', 'sum'], window_sizes=[5, 4, 5])
forecaster = ForecasterAutoregDirect(
                 regressor       = LinearRegression(),
                 steps           = 2,
                 lags            = None,
                 window_features= rolling,
                 differentiation = 1
             )

y = pd.Series(np.arange(10), name='y')
exog = pd.Series(np.arange(100, 110), name='exog')
exog_predict = pd.Series(np.arange(110, 112), name='exog')
exog_predict.index = pd.RangeIndex(start=10, stop=12, step=1)
forecaster.fit(y=y, exog=exog)

In [30]:
forecaster._create_train_X_y(y=y, exog=exog)

(   roll_mean_5  roll_median_4  roll_sum_5  exog_step_1  exog_step_2
 7          1.0            1.0         5.0        106.0        107.0
 8          1.0            1.0         5.0        107.0        108.0
 9          1.0            1.0         5.0        108.0        109.0,
 {1: 6    1.0
  7    1.0
  8    1.0
  Name: y_step_1, dtype: float64,
  2: 7    1.0
  8    1.0
  9    1.0
  Name: y_step_2, dtype: float64},
 ['exog'],
 ['roll_mean_5', 'roll_median_4', 'roll_sum_5'],
 ['exog'],
 ['roll_mean_5', 'roll_median_4', 'roll_sum_5', 'exog_step_1', 'exog_step_2'],
 {'exog': dtype('int64')})

In [93]:
forecaster.create_predict_X(exog=exog_predict)

['roll_mean_5', 'roll_median_4', 'roll_sum_5', 'exog']


,roll_mean_5,roll_median_4,roll_sum_5,exog
10,7.0,7.5,35.0,110.0
11,7.0,7.5,35.0,111.0


In [95]:
forecaster.predict(exog=exog_predict)

10    10.0
11    11.0
Name: pred, dtype: float64

In [35]:
rolling.transform_batch(y)

,roll_mean_4,roll_median_4,roll_sum_5
5,2.5,2.5,10.0
6,3.5,3.5,15.0
7,4.5,4.5,20.0
8,5.5,5.5,25.0
9,6.5,6.5,30.0


In [96]:
y = pd.Series(np.arange(5), name='y')
forecaster = ForecasterAutoregDirect(LinearRegression(), lags=3, steps=2)

forecaster.create_train_X_y(y=y)

(   lag_1  lag_2  lag_3
 4    2.0    1.0    0.0,
 {1: 3    3.0
  Name: y_step_1, dtype: float64,
  2: 4    4.0
  Name: y_step_2, dtype: float64})

In [93]:
from skforecast.preprocessing import RollingFeatures
from skforecast.preprocessing import TimeSeriesDifferentiator
from skforecast.ForecasterAutoreg import ForecasterAutoreg

# Fixtures
from skforecast.ForecasterAutoreg.tests.fixtures_ForecasterAutoreg import y
from skforecast.ForecasterAutoreg.tests.fixtures_ForecasterAutoreg import exog
from skforecast.ForecasterAutoreg.tests.fixtures_ForecasterAutoreg import exog_predict
from skforecast.ForecasterAutoreg.tests.fixtures_ForecasterAutoreg import data  # to test results when using differentiation


In [19]:
y_datetime = y.copy()
y_datetime.index = pd.date_range(start='2001-01-01', periods=len(y), freq='D')
exog_datetime = exog.copy()
exog_datetime.index = pd.date_range(start='2001-01-01', periods=len(exog), freq='D')
exog_predict_datetime = exog_predict.copy()
exog_predict_datetime.index = pd.date_range(start='2001-02-20', periods=len(exog_predict), freq='D')
rolling = RollingFeatures(stats=['mean', 'sum'], window_sizes=3)
forecaster = ForecasterAutoreg(
    LGBMRegressor(verbose=-1, random_state=123), lags=3, window_features=rolling,
    differentiation=1
)
forecaster.fit(y=y_datetime, exog=exog_datetime)

In [ ]:

results = forecaster.predict_bootstrapping(
    steps=4, n_boot=10, exog=exog_predict_datetime, use_in_sample_residuals=True
)
results

In [20]:
y_datetime = pd.Series(
    np.arange(15), index=pd.date_range('2000-01-01', periods=15, freq='D'),
    name='y', dtype=float
)
exog_datetime = pd.Series(
    np.arange(100, 115), index=pd.date_range('2000-01-01', periods=15, freq='D'),
    name='exog', dtype=float
)
rolling = RollingFeatures(
    stats=['mean', 'median', 'sum'], window_sizes=[5, 5, 6]
)

forecaster = ForecasterAutoregDirect(
    LinearRegression(), steps=2, lags=5, window_features=rolling
)
results = forecaster._create_train_X_y(y=y_datetime, exog=exog_datetime)
results

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14.]
0 -1
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13.]
DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07', '2000-01-08',
               '2000-01-09', '2000-01-10', '2000-01-11', '2000-01-12',
               '2000-01-13', '2000-01-14'],
              dtype='datetime64[ns]', freq='D')
DatetimeIndex(['2000-01-08', '2000-01-09', '2000-01-10', '2000-01-11',
               '2000-01-12', '2000-01-13', '2000-01-14', '2000-01-15'],
              dtype='datetime64[ns]', freq='D')


(            lag_1  lag_2  lag_3  lag_4  lag_5  roll_mean_5  roll_median_5  \
 2000-01-08    5.0    4.0    3.0    2.0    1.0          3.0            3.0   
 2000-01-09    6.0    5.0    4.0    3.0    2.0          4.0            4.0   
 2000-01-10    7.0    6.0    5.0    4.0    3.0          5.0            5.0   
 2000-01-11    8.0    7.0    6.0    5.0    4.0          6.0            6.0   
 2000-01-12    9.0    8.0    7.0    6.0    5.0          7.0            7.0   
 2000-01-13   10.0    9.0    8.0    7.0    6.0          8.0            8.0   
 2000-01-14   11.0   10.0    9.0    8.0    7.0          9.0            9.0   
 2000-01-15   12.0   11.0   10.0    9.0    8.0         10.0           10.0   
 
             roll_sum_6  exog_step_1  exog_step_2  
 2000-01-08        15.0        106.0        107.0  
 2000-01-09        21.0        107.0        108.0  
 2000-01-10        27.0        108.0        109.0  
 2000-01-11        33.0        109.0        110.0  
 2000-01-12        39.0        110.0

In [24]:
y_datetime = pd.Series(
    [25.3, 29.1, 27.5, 24.3, 2.1, 46.5, 31.3, 87.1, 133.5, 4.3],
    index=pd.date_range('2000-01-01', periods=10, freq='D'),
    name='y', dtype=float
)
exog = pd.DataFrame({
            'col_1': [7.5, 24.4, 60.3, 57.3, 50.7, 41.4, 87.2, 47.4, 14.6, 73.5],
            'col_2': ['a', 'a', 'a', 'a', 'a', 'b', 'b', 'b', 'b', 'b']},
            index = pd.date_range('2000-01-01', periods=10, freq='D')
        )

transformer_y = StandardScaler()
transformer_exog = ColumnTransformer(
                        [('scale', StandardScaler(), ['col_1']),
                            ('onehot', OneHotEncoder(), ['col_2'])],
                        remainder = 'passthrough',
                        verbose_feature_names_out = False
                    )
rolling = RollingFeatures(
    stats=['ratio_min_max', 'median'], window_sizes=4
)

forecaster = ForecasterAutoregDirect(
                    LinearRegression(), 
                    steps            = 2,
                    lags             = [1, 5], 
                    window_features  = rolling,
                    transformer_y    = transformer_y,
                    transformer_exog = transformer_exog,
                    differentiation  = 1
                )
results = forecaster._create_train_X_y(y=y_datetime, exog=exog)
results

[        nan  0.09974117 -0.04199628 -0.08399257 -0.58269844  1.16539688
 -0.3989647   1.46462041  1.21789224 -3.39119993]
1 -1
[ 0.09974117 -0.04199628 -0.08399257 -0.58269844  1.16539688 -0.3989647
  1.46462041  1.21789224]
DatetimeIndex(['2000-01-02', '2000-01-03', '2000-01-04', '2000-01-05',
               '2000-01-06', '2000-01-07', '2000-01-08', '2000-01-09'],
              dtype='datetime64[ns]', freq='D')
DatetimeIndex(['2000-01-08', '2000-01-09', '2000-01-10'], dtype='datetime64[ns]', freq='D')


(               lag_1     lag_5  roll_ratio_min_max_4  roll_median_4  \
 2000-01-08  1.165397  0.099741             -0.500000      -0.062994   
 2000-01-09 -0.398965 -0.041996             -0.500000      -0.241479   
 2000-01-10  1.464620 -0.083993             -0.397849       0.383216   
 
             col_1_step_1  col_2_a_step_1  col_2_b_step_1  col_1_step_2  \
 2000-01-08      1.698160             0.0             1.0      0.040403   
 2000-01-09      0.040403             0.0             1.0     -1.325790   
 2000-01-10     -1.325790             0.0             1.0      1.127525   
 
             col_2_a_step_2  col_2_b_step_2  
 2000-01-08             0.0             1.0  
 2000-01-09             0.0             1.0  
 2000-01-10             0.0             1.0  ,
 {1: 2000-01-07   -0.398965
  2000-01-08    1.464620
  2000-01-09    1.217892
  Freq: D, Name: y_step_1, dtype: float64,
  2: 2000-01-08    1.464620
  2000-01-09    1.217892
  2000-01-10   -3.391200
  Freq: D, Name: y_step_

In [30]:
results[1][2].to_numpy()

array([ 1.46462041,  1.21789224, -3.39119993])

In [17]:
forecaster = ForecasterAutoregDirect(
                 regressor       = LinearRegression(),
                 steps           = 5,
                 lags            = 100,
                 window_features = None,
                 differentiation = None
             )

y = pd.Series(np.arange(10000), name='y')
output_new, _ = forecaster._create_lags_new(y=y)
output_old, _  = forecaster._create_lags(y=y)

In [18]:
cols = [f"lag_{i}" for i in forecaster.lags]

In [19]:
%%timeit

pd.DataFrame(
    data    = output_new,
    columns = cols
)

37.1 μs ± 384 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [20]:
%%timeit

pd.DataFrame(
    data    = output_old,
    columns = cols
)

37.1 μs ± 306 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [32]:
y = np.arange(10)
y

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [34]:
steps = 2

np.arange(10)[1:len(y)-(steps - 1)]

array([1, 2, 3, 4, 5, 6, 7, 8])

In [35]:
import re
import pytest
import numpy as np
import pandas as pd
from sklearn.exceptions import NotFittedError
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor

from skforecast.preprocessing import RollingFeatures
from skforecast.preprocessing import TimeSeriesDifferentiator
from skforecast.ForecasterAutoreg import ForecasterAutoreg

In [36]:
forecaster = ForecasterAutoreg(
                 regressor       = LinearRegression(),
                 lags            = 20,
                 window_features = None,
                 differentiation = None
             )

y = pd.Series(np.arange(10000), name='y')
forecaster.fit(y=y)

In [44]:
%%timeit

forecaster.predict_bootstrapping(steps=25, n_boot=500)

545 ms ± 4.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [45]:
%%timeit

forecaster.predict_bootstrapping_old(steps=25, n_boot=500)

543 ms ± 2.31 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [48]:
diferenciator = TimeSeriesDifferentiator(order=1)
data_diff = diferenciator.fit_transform(data.to_numpy())
data_diff = pd.Series(data_diff, index=data.index).dropna()
# Simulated exogenous variable
rng = np.random.default_rng(9876)
exog = pd.Series(
    rng.normal(loc=0, scale=1, size=len(data)), index=data.index, name='exog'
)
exog_diff = exog.iloc[1:]
end_train = '2003-03-01 23:59:00'

In [59]:
forecaster_1 = ForecasterAutoregDirect(regressor=LinearRegression(), steps=2, lags=3)
forecaster_1.fit(y=data_diff.loc[:end_train], exog=exog_diff.loc[:end_train])
predictions_diff = forecaster_1.predict(exog=exog_diff.loc[end_train:])
predictions_diff

2003-04-01    0.084479
2003-05-01    0.013194
Freq: MS, Name: pred, dtype: float64

In [60]:
forecaster_2 = ForecasterAutoregDirect(regressor=LinearRegression(), steps=2, lags=3, differentiation=1)
forecaster_2.fit(y=data.loc[:end_train], exog=exog.loc[:end_train])
predictions_2 = forecaster_2.predict(exog=exog.loc[end_train:])
predictions_2

2003-04-01    0.084479
2003-05-01    0.013194
Freq: MS, Name: pred, dtype: float64

In [69]:
y_datetime = pd.Series(
    np.arange(20), index=pd.date_range('2000-01-01', periods=20, freq='D'),
    name='y', dtype=float
)
exog_datetime = pd.DataFrame(
    {'exog_1': np.arange(100, 120), 
        'exog_2': np.arange(200, 220)},
    index=pd.date_range('2000-01-01', periods=20, freq='D')
)
exog_datetime_pred = pd.DataFrame(
    {'exog_1': np.arange(120, 130),
        'exog_2': np.arange(220, 230)},
    index=pd.date_range('2000-01-21', periods=10, freq='D')
)
rolling = RollingFeatures(stats=['mean', 'median'], window_sizes=[5, 5])
rolling_2 = RollingFeatures(stats='sum', window_sizes=[6])

forecaster = ForecasterAutoregDirect(
    LinearRegression(), steps=10, lags=5, window_features=[rolling, rolling_2]
)
forecaster.fit(y=y_datetime, exog=exog_datetime)
results = forecaster.create_predict_X(exog=exog_datetime_pred)
results

,lag_1,lag_2,lag_3,lag_4,lag_5,roll_mean_5,roll_median_5,roll_sum_6,exog_1,exog_2
2000-01-21,19.0,18.0,17.0,16.0,15.0,17.0,17.0,99.0,120.0,220.0
2000-01-22,19.0,18.0,17.0,16.0,15.0,17.0,17.0,99.0,121.0,221.0
2000-01-23,19.0,18.0,17.0,16.0,15.0,17.0,17.0,99.0,122.0,222.0
2000-01-24,19.0,18.0,17.0,16.0,15.0,17.0,17.0,99.0,123.0,223.0
2000-01-25,19.0,18.0,17.0,16.0,15.0,17.0,17.0,99.0,124.0,224.0
2000-01-26,19.0,18.0,17.0,16.0,15.0,17.0,17.0,99.0,125.0,225.0
2000-01-27,19.0,18.0,17.0,16.0,15.0,17.0,17.0,99.0,126.0,226.0
2000-01-28,19.0,18.0,17.0,16.0,15.0,17.0,17.0,99.0,127.0,227.0
2000-01-29,19.0,18.0,17.0,16.0,15.0,17.0,17.0,99.0,128.0,228.0
2000-01-30,19.0,18.0,17.0,16.0,15.0,17.0,17.0,99.0,129.0,229.0


In [88]:
diferenciator = TimeSeriesDifferentiator(order=1)
data_diff = diferenciator.fit_transform(data.to_numpy())
data_diff = pd.Series(data_diff, index=data.index).dropna()

# Simulated exogenous variable
rng = np.random.default_rng(9876)
exog = pd.Series(
    rng.normal(loc=0, scale=1, size=len(data)), index=data.index, name='exog'
)
exog_diff = exog.iloc[1:]
end_train = '2003-03-01 23:59:00'

forecaster_1 = ForecasterAutoregDirect(regressor=LinearRegression(), steps=1, lags=15)
forecaster_1.fit(y=data_diff.loc[:end_train], exog=exog_diff.loc[:end_train])
boot_predictions_diff = forecaster_1.predict_bootstrapping(
    exog=exog_diff.loc[end_train:], n_boot=10
)
boot_predictions_diff

[0.01761623]
[[ 0.10992     0.01980385  0.10648701  0.02050036 -0.00953575  0.07800467
   0.01866525  0.00693296  0.00718461  0.06564146]]


,pred_boot_0,pred_boot_1,pred_boot_2,pred_boot_3,pred_boot_4,pred_boot_5,pred_boot_6,pred_boot_7,pred_boot_8,pred_boot_9
2003-04-01,0.10992,0.019804,0.106487,0.0205,-0.009536,0.078005,0.018665,0.006933,0.007185,0.065641


In [89]:
data.loc[:end_train]

1991-07-01    0.429795
1991-08-01    0.410906
1991-09-01    0.452159
1991-10-01    0.522543
1991-11-01    0.542369
                ...   
2002-11-01    2.366796
2002-12-01    2.472776
2003-01-01    2.458145
2003-02-01    1.968296
2003-03-01    2.043333
Freq: MS, Name: y, Length: 141, dtype: float64

In [90]:
last_value_train = data.loc[:end_train].iloc[[-1]]
print(last_value_train)
boot_predictions_1 = boot_predictions_diff.copy()
boot_predictions_1.loc[last_value_train.index[0]] = last_value_train.values[0]
boot_predictions_1 = boot_predictions_1.sort_index()
boot_predictions_1 = boot_predictions_1.cumsum(axis=0).iloc[1:,]
boot_predictions_1

2003-03-01    2.043333
Freq: MS, Name: y, dtype: float64


,pred_boot_0,pred_boot_1,pred_boot_2,pred_boot_3,pred_boot_4,pred_boot_5,pred_boot_6,pred_boot_7,pred_boot_8,pred_boot_9
2003-04-01,2.153253,2.063137,2.14982,2.063834,2.033798,2.121338,2.061999,2.050266,2.050518,2.108975


In [91]:
forecaster_2 = ForecasterAutoregDirect(regressor=LinearRegression(), steps=1, lags=15, differentiation=1)
forecaster_2.fit(y=data.loc[:end_train], exog=exog.loc[:end_train])
boot_predictions_2 = forecaster_2.predict_bootstrapping(
        exog=exog_diff.loc[end_train:], n_boot=10
    )
boot_predictions_2.head(3)

[0.01761623]
[[ 0.10992     0.01980385  0.10648701  0.02050036 -0.00953575  0.07800467
   0.01866525  0.00693296  0.00718461  0.06564146]]


,pred_boot_0,pred_boot_1,pred_boot_2,pred_boot_3,pred_boot_4,pred_boot_5,pred_boot_6,pred_boot_7,pred_boot_8,pred_boot_9
2003-04-01,2.153253,2.063137,2.14982,2.063834,2.033798,2.121338,2.061999,2.050266,2.050518,2.108975


In [86]:
forecaster_2 = ForecasterAutoregDirect(regressor=LinearRegression(), steps=1, lags=15, differentiation=1)
forecaster_2.fit(y=data.loc[:end_train], exog=exog.loc[:end_train])
forecaster_2.in_sample_residuals_

{1: array([ 0.01746861,  0.09230377,  0.04060913, -0.05912942, -0.00600831,
        -0.021614  ,  0.00288413, -0.00272942, -0.01943126,  0.02912783,
         0.00733195,  0.0242183 ,  0.03504128,  0.1372057 , -0.0080244 ,
        -0.05878831,  0.00982868,  0.02356983,  0.02241519,  0.00518851,
         0.00406041,  0.04802523,  0.00565868, -0.01068326, -0.04404133,
        -0.183798  ,  0.00468954,  0.06038844,  0.0935621 ,  0.00894786,
        -0.01337187,  0.00104902,  0.02223371,  0.02324297, -0.02371574,
         0.06415126,  0.01600326,  0.04952103,  0.06624363, -0.02478322,
        -0.06424415, -0.01043162,  0.02009803, -0.04190381,  0.03090841,
        -0.05712068, -0.03306857, -0.03877012, -0.02371599,  0.03598932,
        -0.0741605 , -0.06631351, -0.04751757,  0.00874729, -0.02114579,
         0.02030593, -0.02565921, -0.04489306,  0.0068321 ,  0.0125604 ,
        -0.03625581,  0.01943021, -0.04010963,  0.02787261,  0.03436609,
        -0.00655551, -0.0595945 , -0.00951804, -

In [87]:
forecaster_2 = ForecasterAutoregDirect(regressor=LinearRegression(), steps=1, lags=15)
forecaster_2.fit(y=data.loc[:end_train], exog=exog.loc[:end_train])
forecaster_2.in_sample_residuals_

{1: array([ 1.93587251e-02,  1.26802694e-02,  8.44971510e-02,  3.27958537e-02,
        -6.06432335e-02, -1.39289181e-02, -3.87806315e-02, -3.13277467e-03,
        -1.19317174e-02, -2.85791372e-02,  2.22898821e-02, -2.04541278e-03,
         1.45928262e-02,  3.23365608e-02,  1.29325483e-01, -5.45830942e-03,
        -5.82717439e-02,  8.26023955e-04,  3.88679728e-03,  2.01419352e-02,
        -1.33320108e-03, -3.89320349e-03,  4.20044264e-02, -6.24842683e-04,
        -1.55812039e-02, -4.75101057e-02, -1.95007513e-01, -1.29086147e-02,
         4.30641169e-02,  8.19665994e-02,  3.04101552e-02, -2.67473592e-03,
        -5.14338253e-03,  1.74966832e-02,  1.73502558e-02, -2.69133857e-02,
         6.20363201e-02,  1.20481378e-02,  4.90844376e-02,  7.16867054e-02,
        -3.08400458e-02, -6.36030704e-02, -1.84767305e-02,  1.10726191e-02,
        -4.81551685e-02,  3.30818218e-02, -6.09936728e-02, -3.85385280e-02,
        -4.03152874e-02, -3.73337491e-02,  3.69862191e-02, -7.57552091e-02,
        -

In [96]:
y_datetime = y.copy()
y_datetime.index = pd.date_range(start='2001-01-01', periods=len(y), freq='D')
exog_datetime = exog.copy()
exog_datetime.index = pd.date_range(start='2001-01-01', periods=len(exog), freq='D')
exog_predict_datetime = exog_predict.copy()
exog_predict_datetime.index = pd.date_range(start='2001-02-20', periods=len(exog_predict), freq='D')
rolling = RollingFeatures(stats=['mean', 'sum'], window_sizes=[3, 5])
forecaster = ForecasterAutoregDirect(
    LGBMRegressor(verbose=-1, random_state=123), steps=10, lags=3, window_features=rolling
)
forecaster.fit(y=y_datetime, exog=exog_datetime)
results = forecaster.predict_bootstrapping(
    n_boot=10, exog=exog_predict_datetime, use_in_sample_residuals=True
)
results

,pred_boot_0,pred_boot_1,pred_boot_2,pred_boot_3,pred_boot_4,pred_boot_5,pred_boot_6,pred_boot_7,pred_boot_8,pred_boot_9
2001-02-20,0.423106,0.630976,0.361789,0.980764,0.893389,0.438572,0.398044,0.729050,0.175452,0.729050
2001-02-21,0.531551,0.312261,0.724455,0.501837,0.724455,0.737995,0.425830,0.312261,0.893389,0.944160
2001-02-22,0.684830,0.322959,0.182492,0.737995,0.737995,0.426351,0.312261,0.398044,0.849432,0.493685
2001-02-23,0.059678,0.092105,0.611024,0.115618,0.425830,0.182492,0.944160,0.426351,0.737995,0.361789
2001-02-24,0.893389,0.175452,0.175452,0.398044,0.392118,0.361789,0.392118,0.630976,0.611024,0.737995
2001-02-25,0.611024,0.343178,0.737995,0.361789,0.438572,0.426351,0.531828,0.414826,0.182492,0.430863
2001-02-26,0.630976,0.866309,0.493685,0.317285,0.228263,0.531551,0.175452,0.317285,0.531551,0.893389
2001-02-27,0.092105,0.724455,0.361789,0.623953,0.293714,0.414826,0.483034,0.724455,0.092105,0.092105
2001-02-28,0.430863,0.737995,0.361789,0.493685,0.483034,0.849432,0.426351,0.623953,0.483034,0.531828
2001-03-01,0.944160,0.322959,0.630976,0.398044,0.623953,0.737995,0.228263,0.433701,0.944160,0.092105


In [97]:
results.to_numpy()

array([[0.42310646, 0.63097612, 0.36178866, 0.9807642 , 0.89338916,
        0.43857224, 0.39804426, 0.72904971, 0.17545176, 0.72904971],
       [0.53155137, 0.31226122, 0.72445532, 0.50183668, 0.72445532,
        0.73799541, 0.42583029, 0.31226122, 0.89338916, 0.94416002],
       [0.68482974, 0.32295891, 0.18249173, 0.73799541, 0.73799541,
        0.42635131, 0.31226122, 0.39804426, 0.84943179, 0.4936851 ],
       [0.0596779 , 0.09210494, 0.61102351, 0.1156184 , 0.42583029,
        0.18249173, 0.94416002, 0.42635131, 0.73799541, 0.36178866],
       [0.89338916, 0.17545176, 0.17545176, 0.39804426, 0.39211752,
        0.36178866, 0.39211752, 0.63097612, 0.61102351, 0.73799541],
       [0.61102351, 0.34317802, 0.73799541, 0.36178866, 0.43857224,
        0.42635131, 0.53182759, 0.41482621, 0.18249173, 0.43086276],
       [0.63097612, 0.86630916, 0.4936851 , 0.31728548, 0.22826323,
        0.53155137, 0.17545176, 0.31728548, 0.53155137, 0.89338916],
       [0.09210494, 0.72445532, 0.3617886